# Proof-of-process only; uses outdated variables and no data. See Notebook 2 for the runnable demo with synthetic data.

# 01 — Build Dataset (Joins + Features)

In [2]:
# Imports
import pandas as pd, numpy as np, re, os
from pathlib import Path


## Load data paths

In [ ]:
DATA_DIR = Path("/Cast_and_Crew/data")
CLEAN = DATA_DIR/'clean'

NETFLIX_TITLES = CLEAN/'netflix_movies_agg.csv'  
NETFLIX_RATINGS = CLEAN/'netflix_ratings.csv'      

BOX_CAST = CLEAN/'Mojo_budget_update.csv'



## Helpers

In [14]:
def norm_title(s):
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9 ]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def zscore_by_year(df, col, year_col='movie_year'):
    df = df.copy()
    df['__log'] = np.log1p(df[col])
    df['__mu'] = df.groupby(year_col)['__log'].transform('mean')
    df['__sd'] = df.groupby(year_col)['__log'].transform('std').replace(0,1)
    df[col + '_z_by_year'] = (df['__log'] - df['__mu']) / df['__sd']
    return df.drop(columns=['__log','__mu','__sd'])


## Load Netflix titles & ratings (aggregate to movie-level)

In [ ]:

titles = pd.read_csv(NETFLIX_TITLES)
# Expect columns like: movie_id, year, title
titles['title_norm'] = titles['movie_title'].map(norm_title)

# Ratings aggregation (update to your schema)
ratings = pd.read_csv(NETFLIX_RATINGS)
# Expect columns like: user_id, movie_id, rating, date
movie_avg = ratings.groupby('movie_id')['rating'].agg(['mean','count']).reset_index()
movie_avg.columns = ['movie_id','movie_avg_rating','movie_rating_count']

movies = titles.merge(movie_avg, on='movie_id', how='left')
movies = movies.rename(columns={'year':'movie_year', 'title':'movie_title'})


## Load Box Office + Cast

In [19]:
box = pd.read_csv(BOX_CAST)
box['title_norm'] = box['title'].map(norm_title)


In [21]:
# 🔧 Harmonize column names before merge
def norm_title(s: str) -> str:
    import re
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9 ]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Coalesce year + title columns on both sides
def coalesce_year_title(df):
    # year -> movie_year
    for cand in ["movie_year", "year", "startYear", "release_year", "Year"]:
        if cand in df.columns:
            df = df.rename(columns={cand: "movie_year"})
            break
    # title -> movie_title
    for cand in ["movie_title", "title", "primaryTitle", "MovieTitle"]:
        if cand in df.columns:
            df = df.rename(columns={cand: "movie_title"})
            break
    # normalize types and title_norm
    if "movie_year" in df.columns:
        df["movie_year"] = pd.to_numeric(df["movie_year"], errors="coerce").astype("Int64")
    if "movie_title" in df.columns:
        df["title_norm"] = df["movie_title"].map(norm_title)
    return df

movies = coalesce_year_title(movies)
box    = coalesce_year_title(box)

# drop rows missing year or title_norm
movies = movies.dropna(subset=["movie_year", "title_norm"])
box    = box.dropna(subset=["movie_year", "title_norm"])

# quick sanity
print("movies cols:", sorted(movies.columns))
print("box cols:", sorted(box.columns))
print("year dtype (movies, box):", movies["movie_year"].dtype, box["movie_year"].dtype)


movies cols: ['movie_avg_rating_x', 'movie_avg_rating_y', 'movie_id', 'movie_rating_count_x', 'movie_rating_count_y', 'movie_title', 'movie_year', 'title_norm']
box cols: ['budget', 'cinematographer', 'composer', 'director', 'distributor', 'domestic', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html', 'international', 'main_actor_1', 'main_actor_2', 'main_actor_3', 'main_actor_4', 'movie_id', 'movie_title', 'movie_year', 'mpaa', 'producer', 'release_date', 'run_time', 'title_norm', 'trivia', 'worldwide', 'writer']
year dtype (movies, box): Int64 Int64


## Join on (title_norm, movie_year)

In [22]:
mv = movies.merge(box, left_on=['title_norm','movie_year'],
                        right_on=['title_norm','movie_year'], how='inner', suffixes=('','_box'))
print('Joined rows:', len(mv))


Joined rows: 1147


## Build features (Meehl 10)

In [28]:
df = mv.copy()
from pathlib import Path

# Budget/box normalization
for col in ['worldwide','domestic','budget']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=['worldwide'])  # keep rows with outcome
df = zscore_by_year(df, 'worldwide', year_col='movie_year')


def ensure_movie_avg_rating(movies: pd.DataFrame) -> pd.DataFrame:
    # If the right column already exists, just return
    if 'movie_avg_rating' in movies.columns:
        return movies

    # Try to find a "mean" column from an earlier aggregate
    mean_aliases  = {'movie_avg_rating','avg_rating','mean_rating','rating_mean','mean'}
    count_aliases = {'movie_rating_count','rating_count','count','n'}
    mean_col = next((c for c in movies.columns if c.lower() in mean_aliases), None)
    count_col = next((c for c in movies.columns if c.lower() in count_aliases), None)

    if mean_col:
        movies = movies.rename(columns={mean_col: 'movie_avg_rating'})
        if count_col:
            movies = movies.rename(columns={count_col: 'movie_rating_count'})
        return movies

    # Fallback: compute from ratings file if available
    ratings_path = CLEAN / 'netflix_ratings.csv'
    if ratings_path.exists():
        rat = pd.read_csv(ratings_path, usecols=['movie_id','rating'])
        agg = rat.groupby('movie_id')['rating'].agg(['mean','count']).reset_index()
        agg.columns = ['movie_id','movie_avg_rating','movie_rating_count']
        movies = movies.merge(agg, on='movie_id', how='left')
        return movies

    # Last resort: impute a neutral average so you can proceed (you can replace later)
    movies['movie_avg_rating'] = 3.5
    movies['movie_rating_count'] = 0
    return movies

# 1) Standardize year/title on both tables (safe if already done)
def norm_title(s):
    import re
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9 ]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def coalesce_year_title(df):
    for cand in ["movie_year","year","startYear","release_year","Year"]:
        if cand in df.columns:
            df = df.rename(columns={cand: "movie_year"})
            break
    for cand in ["movie_title","title","primaryTitle","MovieTitle"]:
        if cand in df.columns:
            df = df.rename(columns={cand: "movie_title"})
            break
    if "movie_title" in df.columns:
        df["title_norm"] = df["movie_title"].map(norm_title)
    if "movie_year" in df.columns:
        df["movie_year"] = pd.to_numeric(df["movie_year"], errors="coerce").astype("Int64")
    return df

movies = coalesce_year_title(movies)
box    = coalesce_year_title(box)

# 2) Make sure movies has movie_avg_rating
movies = ensure_movie_avg_rating(movies)

# 3) Re-do the join cleanly
movies = movies.dropna(subset=["movie_year","title_norm"])
box    = box.dropna(subset=["movie_year","title_norm"])

mv = movies.merge(
    box,
    on=["title_norm","movie_year"],
    how="inner",
    suffixes=("","_box")
)
print("Joined rows:", len(mv))

# ---- from here on, work on a fresh df built from mv ----
df = mv.copy()

# Ensure money columns are numeric
for col in ["worldwide","domestic","international","budget"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Make sure we actually have worldwide
assert "worldwide" in df.columns, \
    f"'worldwide' not found. Got money-like cols: {[c for c in df.columns if 'world' in c.lower() or 'gross' in c.lower()]}"

# Drop rows without outcome and recompute the z-score AFTER the merge
df = df.dropna(subset=["worldwide"]).copy()

# helper if you don't already have it defined above
def zscore_by_year(_df, col, year_col="movie_year"):
    d = _df.copy()
    logv = np.log1p(pd.to_numeric(d[col], errors="coerce"))
    mu = d.groupby(year_col)[col].transform(lambda s: np.log1p(pd.to_numeric(s, errors="coerce")).mean())
    sd = d.groupby(year_col)[col].transform(lambda s: np.log1p(pd.to_numeric(s, errors="coerce")).std()).replace(0,1)
    d[f"{col}_z_by_year"] = (logv - mu) / sd
    return d

df = zscore_by_year(df, "worldwide", year_col="movie_year")

# Ensure movie_avg_rating exists (rename or compute from ratings if needed)
if "movie_avg_rating" not in df.columns:
    mean_aliases  = {"movie_avg_rating","avg_rating","mean_rating","rating_mean","mean"}
    count_aliases = {"movie_rating_count","rating_count","count","n"}
    mean_col = next((c for c in df.columns if c.lower() in mean_aliases), None)
    count_col = next((c for c in df.columns if c.lower() in count_aliases), None)
    if mean_col:
        df = df.rename(columns={mean_col:"movie_avg_rating"})
        if count_col:
            df = df.rename(columns={count_col:"movie_rating_count"})
    else:
        # fallback: compute from ratings file if present
        ratings_path = CLEAN/"netflix_ratings.csv"
        if ratings_path.exists():
            rat = pd.read_csv(ratings_path, usecols=["movie_id","rating"])
            agg = rat.groupby("movie_id")["rating"].agg(["mean","count"]).reset_index()
            agg.columns = ["movie_id","movie_avg_rating","movie_rating_count"]
            df = df.merge(agg, on="movie_id", how="left")
        else:
            df["movie_avg_rating"] = 3.5
            df["movie_rating_count"] = 0

# Scale ratings and build resonance
rmin, rmax = df["movie_avg_rating"].min(), df["movie_avg_rating"].max()
df["rating_01"] = (df["movie_avg_rating"] - rmin) / (rmax - rmin + 1e-9)
df["resonance"] = 0.5*df["worldwide_z_by_year"] + 0.5*df["rating_01"]

# Genre one-hot (limited set for MVP)
for g in ["Action","Drama","Comedy","Thriller","Romance"]:
    df[f"genre_{g.lower()}"] = (
        df[["genre_1","genre_2","genre_3","genre_4"]]
          .astype(str).apply(lambda r: int(any(g.lower() in x.lower() for x in r.values)), axis=1)
    )

# Year cyclicals
year = df["movie_year"].astype(float)
span = (year.max() - year.min()) + 1.0
df["year_sin"] = np.sin(2*np.pi*(year - year.min())/span)
df["year_cos"] = np.cos(2*np.pi*(year - year.min())/span)

# (The rest of your placeholders / saves stay the same)

# Top-billing propensity (for attached cast)
for i in [1,2,3,4]:
    df[f'main_actor_{i}'] = df.get(f'main_actor_{i}', pd.Series([None]*len(df)))

# Actor/Director trailing stats -- placeholders (fill with rolling over prior roles)
# For MVP, back off to global priors; you can implement true trailing via a per-actor filmography join.
df['atr3'] = df['worldwide_z_by_year'].expanding().mean() * 0  # TODO: replace with actor-level trailing
df['dtr3'] = df['worldwide_z_by_year'].expanding().mean() * 0  # TODO: replace with director-level trailing

# Budget fit delta (deciles)
df['budget_decile'] = pd.qcut(df['budget'].fillna(df['budget'].median()), 10, labels=False, duplicates='drop')
df['project_budget_decile'] = df['budget_decile']  # proxy in MVP
df['bfd'] = 0  # will be |project - actor_budget_decile| when actor stats are computed

# Release velocity (proxy)
df['release_density'] = 1  # TODO
df['recency_gap'] = 0      # TODO

# MPAA fit (proxy zeros for MVP)
df['mpaa_fit'] = 0

# Save modeling table
MODEL_CSV = CLEAN/'modeling_table_mvp.csv'
df.to_csv(MODEL_CSV, index=False)
print('Saved', MODEL_CSV)


Joined rows: 1147
Saved \Cast_and_Crew\data\clean\modeling_table_mvp.csv
